In [ ]:
import requests
import random
import json

In [ ]:
base_url = "http://localhost:8081"
keycloak_url = "http://keycloak:8080"

In [ ]:
resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"

In [ ]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [ ]:
swaggerJson = resp.json()
swaggerJson["paths"].keys()

In [ ]:
resp = requests.get("%s%s"%(base_url, '/api/docs/Student.json'))
assert resp.status_code == 200
resp.json()

In [ ]:
resp = requests.get("%s%s"%(base_url, '/api/docs/Teacher.json'))
assert resp.status_code == 200
resp.json()

# invite student

In [ ]:
userId =str(random.randint(1e10,1e11))
inviteJson = {
    'name': 'u1',
    'mobile': userId
}
resp = requests.post("%s%s"%(base_url, '/api/v1/Student/invite'), json=inviteJson)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Student"==entity_name
idx = resp.json()["result"][entity_name]["osid"]
print(idx)

In [ ]:
idx

In [ ]:
userId

In [ ]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}

data = {
  'client_id': 'registry-frontend',
  'username': userId,
  'password': 'abcd@123',
  'grant_type': 'password'
}

response = requests.post('%s/auth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloak_url), headers=headers, data=data)
print(response.json())
print(response.status_code)
token = response.json()["access_token"]


In [ ]:
token


In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get("%s%s"%(base_url, '/api/v1/Student'), headers=headers,)
print("%s%s"%(base_url, '/api/v1/Student/'))
response.text
# response.status_code, response.json()

## Create teacher

In [ ]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

data = '{"name": "teacher1", "mobile": "%s"}'%userId
print(data)
response = requests.post('http://localhost:8081/api/v1/Teacher', headers=headers, data=data)

In [ ]:
response.status_code, response.json()

In [ ]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher', headers=headers)
assert response.status_code == 200
print(response.json())
response.json()